# 2023_04_24

기존의 데이터를 이런식으로 `EventId`를 기준으로 데이터를 생성했는데 이게 오류가 있어서 수정함  

```python
for e_id in tqdm(weather_city_df['EventId'],desc =f'{city} weather'):
    weather_event_df = weather_city_df[weather_city_df['EventId'] == e_id][weather_insert_columns]
    dt = pd.date_range(weather_event_df['OccTime'].iloc[0], weather_event_df['EndTime'].iloc[0], freq='1h')
    for d in dt:
        #print(d)
        weather_count_df = weather_count_df.append(weather_event_df.iloc[0], ignore_index=True)

weather_count_df = weather_count_df[weather_count_df['OccTime'].notnull()]
```

수정사항  

```python
for e_id in tqdm(city_df['EventId'], desc = f'{city} Traffic'):
    event_df = city_df[city_df['EventId'] == e_id][insert_columns]
    event_df.reset_index(inplace=True , drop=True)
    event_df_first_row = event_df.iloc[0].copy()
    dt = pd.date_range(event_df['OccTime'].iloc[0], event_df['EndTime'].iloc[0], freq='1h')
    for dt_idx , d in tqdm(enumerate(dt)):
        event_df.loc[dt_idx] = event_df.loc[0]
        event_df.loc[dt_idx, 'OccTime'] = d
        count_df = count_df.append(event_df, ignore_index=True)
        
        new_row = event_df_first_row.copy()
        new_row['OccTime'] = d
        count_df = count_df.append(new_row, ignore_index=True)
count_df = count_df[count_df['OccTime'].notnull()]
```

다음과 같이 코드를 고쳤으나 시간이 너무 오래 걸리는 관계로 mulit-thread를 이용해서 수행시간 단축  

``` python
from joblib import Parallel, delayed
from tqdm import tqdm

insert_columns = ['OccTime', 'EndTime', 'GeoHash', 'Type', 'Severity', 'EventId', 'Distance(mi)', 'Description']
count_df = pd.DataFrame(columns=insert_columns)

def process_event(e_id):
    event_df = city_df[city_df['EventId'] == e_id][insert_columns]
    event_df.reset_index(inplace=True , drop=True)
    event_df_first_row = event_df.iloc[0].copy()
    dt = pd.date_range(event_df['OccTime'].iloc[0], event_df['EndTime'].iloc[0], freq='1h')
    rows = []
    for dt_idx , d in enumerate(dt):
        event_df.loc[dt_idx] = event_df.loc[0]
        event_df.loc[dt_idx, 'OccTime'] = d
        rows.append(event_df.copy())

        new_row = event_df_first_row.copy()
        new_row['OccTime'] = d
        rows.append(new_row)

    return pd.concat(rows, ignore_index=True)

results = Parallel(n_jobs=-1)(delayed(process_event)(e_id) for e_id in tqdm(city_df['EventId'], desc = f'{city} Traffic'))

count_df = pd.concat(results, ignore_index=True)
count_df = count_df[count_df['OccTime'].notnull()]
```

중도시 `Long Beach`를 기준으로10h -> 2h로 시간이 줄어들음  